In [5]:
import pandas as pd
import re


# Summary
This presents the evaluation of our approach in a way that is compatible to Chaaben et al.'s work.

## Threats to validity
We see a large homogenity in the data (a lot of EStringToStringMapEntries with key documentation)

## Differences between the original approach and our implementation and evaluation
- (-) We do not apply the approach to UML classdiagram
- (+) We apply the approach to Ecore diagrams in a way that is compatible to class diagrams (for add concept, add relationship comparison). Add relationship we evaluate many-fold. First, in Ecore add "EReference" can be interpreted, roughly speaking, to adding associations in class diagrams. There are some occurrences of this in our ``add concept'' evlaution. Furthermore, we can evaluate if the correct edge (source, target nodes, directions, and association name), is correct in scenarios where a Reference is added in Ecore.
- (-) We do not use a large random sample and rank
- (+) We use the simple change graph, wich already focuses on the changing part
- (-) We do not use GPT-3
- (+) We do use GPT-4

## Some words on the comparison
Chaaben et al.' approach is more comparable to a random retrieval of few-shot samples (with a fixed set of samples). 
Furthermore, our approach reflects the graph like nature of models while Chaaben et al.'s approach (for new concept recommendation) utilizes random pairs of concepts which not necesarily reflect related concepts in the original models.
In this sense, our approach is superior to Chaaben et al.'s approach and therefore shows a superior performance.
Anyway, this is not a shortcoming of Chaaben et al.'s approach and instead rather a consequence of their approach beeing an early initial experimentation with few-shot learning.


# Load Data

In [2]:
# Example CSV file:
# ,id,source,prompt,completion,few_shot_token_count,test_token_count,total_token_count,few_shot_count,test_edge_count,test_completion_edge_count,completion_type,total_tokens,completion_tokens,completion_string,correct_format,type_isomorphic_completion,change_type_isomorphic_completion,structural_isomorphic_completion
# 0,0,experiment_samples/technology.cbi!!org.eclipse.b3.backend_model_B3Backend.ecore/test_samples.jsonl,"t # 69
# e 1 0 ""{'changeType': 'Change', 'type': 'attribute'}"" ""{'changeType': 'Preserve', 'type': 'object', 'className': 'EAttribute', 'attributes': {'id': '_5T16md6tEei97MD7GK1RmA', 'name':'id','ordered':'true','unique':'true','lowerBound':'1','upperBound':'1','many':'false','required':'true','eType':'String','eGenericType':'org.eclipse.emf.ecore.impl.EGenericTypeImpl@6a0f4413 (expression: String)','changeable':'true','volatile':'false','transient':'false','unsettable':'false','derived':'false','eContainingClass':'VariableDeclaration','iD':'false','eAttributeType':'String'}}"" ""{'changeType': 'Change', 'type': 'attributeValue', 'className': 'EAttribute', 'attributeName': 'ordered', 'valueBefore': 'true', 'valueAfter': 'false'}""
# e 1 2 ""{'changeType': 'Change', 'type': 'attribute'}"" _ ""{'changeType': 'Change', 'type': 'attributeValue', 'className': 'EAttribute', 'attributeName': 'unique', 'valueBefore': 'true', 'valueAfter': 'false'}""

# $$
# ---
# t # 63
# e 2 1 ""{'changeType': 'Change', 'type': 'attribute'}"" ""{'changeType': 'Preserve', 'type': 'object', 'className': 'EAttribute', 'attributes': {'id': '_5T16Ft6tEei97MD7GK1RmA', 'name':'name','ordered':'true','unique':'true','lowerBound':'1','upperBound':'1','many':'false','required':'true','eType':'String','eGenericType':'org.eclipse.emf.ecore.impl.EGenericTypeImpl@7a674f5b (expression: String)','changeable':'true','volatile':'false','transient':'false','unsettable':'false','derived':'false','eContainingClass':'Unit','iD':'false','eAttributeType':'String'}}"" ""{'changeType': 'Change', 'type': 'attributeValue', 'className': 'EAttribute', 'attributeName': 'ordered', 'valueBefore': 'true', 'valueAfter': 'false'}""
# e 2 0 ""{'changeType': 'Change', 'type': 'attribute'}"" _ ""{'changeType': 'Change', 'type': 'attributeValue', 'className': 'EAttribute', 'attributeName': 'unique', 'valueBefore': 'true', 'valueAfter': 'false'}""

# $$
# ---
# t # 47
# e 0 2 ""{'changeType': 'Add', 'type': 'reference', 'referenceTypeName': 'eOpposite'}"" ""{'changeType': 'Preserve', 'type': 'object', 'className': 'EReference', 'attributes': {'id': '_XMof9960Eei97MD7GK1RmA', 'name':'decorator','ordered':'true','unique':'true','lowerBound':'0','upperBound':'1','many':'false','required':'false','eType':'Decorator','eGenericType':'org.eclipse.emf.ecore.impl.EGenericTypeImpl@70824c2b (expression: Decorator)','changeable':'true','volatile':'false','transient':'true','unsettable':'false','derived':'false','eContainingClass':'DynamicLabel','containment':'false','container':'false','resolveProxies':'false','eOpposite':'labelsToUpdate','eReferenceType':'Decorator'}}"" ""{'changeType': 'Preserve', 'type': 'object', 'className': 'EReference', 'attributes': {'id': '_XMyS0960Eei97MD7GK1RmA', 'name':'labelsToUpdate','ordered':'true','unique':'true','lowerBound':'0','upperBound':'-1','many':'true','required':'false','eType':'DynamicLabel','eGenericType':'org.eclipse.emf.ecore.impl.EGenericTypeImpl@98069e5 (expression: DynamicLabel)','changeable':'true','volatile':'false','transient':'true','unsettable':'false','derived':'false','eContainingClass':'Decorator','containment':'false','container':'false','resolveProxies':'false','eOpposite':'decorator','eReferenceType':'DynamicLabel'}}""
# e 0 1 ""{'changeType': 'Remove', 'type': 'reference', 'referenceTypeName': 'eOpposite'}"" _ ""{'changeType': 'Preserve', 'type': 'object', 'className': 'EReference', 'attributes': {'id': '_XMyS0960Eei97MD7GK1RmA', 'name':'labelsToUpdate','ordered':'true','unique':'true','lowerBound':'0','upperBound':'-1','many':'true','required':'false','eType':'DynamicLabel','eGenericType':'org.eclipse.emf.ecore.impl.EGenericTypeImpl@4667369e (expression: DynamicLabel)','changeable':'true','volatile':'false','transient':'false','unsettable':'false','derived':'false','eContainingClass':'Decorator','containment':'false','container':'false','resolveProxies':'true','eOpposite':'decorator','eReferenceType':'DynamicLabel'}}""

# $$
# ---
# t # 78
# e 0 1 ""{'changeType': 'Change', 'type': 'attribute'}"" ""{'changeType': 'Preserve', 'type': 'object', 'className': 'EAttribute', 'attributes': {'id': '_5T16S96tEei97MD7GK1RmA', 'name':'name','ordered':'true','unique':'true','lowerBound':'1','upperBound':'1','many':'false','required':'true','eType':'String','eGenericType':'org.eclipse.emf.ecore.impl.EGenericTypeImpl@5101b9e8 (expression: String)','changeable':'true','volatile':'false','transient':'false','unsettable':'false','derived':'false','eContainingClass':'OclFeature','iD':'false','eAttributeType':'String'}}"" ""{'changeType': 'Change', 'type': 'attributeValue', 'className': 'EAttribute', 'attributeName': 'unique', 'valueBefore': 'true', 'valueAfter': 'false'}""
# e 0 2 ""{'changeType': 'Change', 'type': 'attribute'}"" _ ""{'changeType': 'Change', 'type': 'attributeValue', 'className': 'EAttribute', 'attributeName': 'ordered', 'valueBefore': 'true', 'valueAfter': 'false'}""

# $$
# ---
# t # 74
# e 0 2 ""{'changeType': 'Change', 'type': 'attribute'}"" ""{'changeType': 'Preserve', 'type': 'object', 'className': 'EAttribute', 'attributes': {'id': '_5T16uN6tEei97MD7GK1RmA', 'name':'name','ordered':'true','unique':'true','lowerBound':'1','upperBound':'1','many':'false','required':'true','eType':'String','eGenericType':'org.eclipse.emf.ecore.impl.EGenericTypeImpl@3a30bf57 (expression: String)','changeable':'true','volatile':'false','transient':'false','unsettable':'false','derived':'false','eContainingClass':'OclType','iD':'false','eAttributeType':'String'}}"" ""{'changeType': 'Change', 'type': 'attributeValue', 'className': 'EAttribute', 'attributeName': 'ordered', 'valueBefore': 'true', 'valueAfter': 'false'}""
# e 0 1 ""{'changeType': 'Change', 'type': 'attribute'}"" _ ""{'changeType': 'Change', 'type': 'attributeValue', 'className': 'EAttribute', 'attributeName': 'unique', 'valueBefore': 'true', 'valueAfter': 'false'}""

# $$
# ---
# t # 22
# e 0 2 ""{'changeType': 'Add', 'type': 'reference', 'referenceTypeName': 'eOpposite'}"" ""{'changeType': 'Preserve', 'type': 'object', 'className': 'EReference', 'attributes': {'id': '_XMof9960Eei97MD7GK1RmA', 'name':'decorator','ordered':'true','unique':'true','lowerBound':'0','upperBound':'1','many':'false','required':'false','eType':'Decorator','eGenericType':'org.eclipse.emf.ecore.impl.EGenericTypeImpl@4dd09c5c (expression: Decorator)','changeable':'true','volatile':'false','transient':'true','unsettable':'false','derived':'false','eContainingClass':'DynamicLabel','containment':'false','container':'false','resolveProxies':'false','eOpposite':'labelsToUpdate','eReferenceType':'Decorator'}}"" ""{'changeType': 'Preserve', 'type': 'object', 'className': 'EReference', 'attributes': {'id': '_XMyS0960Eei97MD7GK1RmA', 'name':'labelsToUpdate','ordered':'true','unique':'true','lowerBound':'0','upperBound':'-1','many':'true','required':'false','eType':'DynamicLabel','eGenericType':'org.eclipse.emf.ecore.impl.EGenericTypeImpl@35139877 (expression: DynamicLabel)','changeable':'true','volatile':'false','transient':'true','unsettable':'false','derived':'false','eContainingClass':'Decorator','containment':'false','container':'false','resolveProxies':'false','eOpposite':'decorator','eReferenceType':'DynamicLabel'}}""
# e 0 1 ""{'changeType': 'Remove', 'type': 'reference', 'referenceTypeName': 'eOpposite'}"" _ ""{'changeType': 'Preserve', 'type': 'object', 'className': 'EReference', 'attributes': {'id': '_XMyS0960Eei97MD7GK1RmA', 'name':'labelsToUpdate','ordered':'true','unique':'true','lowerBound':'0','upperBound':'-1','many':'true','required':'false','eType':'DynamicLabel','eGenericType':'org.eclipse.emf.ecore.impl.EGenericTypeImpl@6dc32004 (expression: DynamicLabel)','changeable':'true','volatile':'false','transient':'true','unsettable':'false','derived':'false','eContainingClass':'Decorator','containment':'false','container':'false','resolveProxies':'false','eOpposite':'decorator','eReferenceType':'DynamicLabel'}}""

# $$
# ---
# t # 32
# e 2 0 ""{'changeType': 'Add', 'type': 'reference', 'referenceTypeName': 'eOpposite'}"" ""{'changeType': 'Preserve', 'type': 'object', 'className': 'EReference', 'attributes': {'id': '_XMogPN60Eei97MD7GK1RmA', 'name':'decorators','ordered':'true','unique':'true','lowerBound':'0','upperBound':'-1','many':'true','required':'false','eType':'Decorator','eGenericType':'org.eclipse.emf.ecore.impl.EGenericTypeImpl@15e6146f (expression: Decorator)','changeable':'true','volatile':'false','transient':'false','unsettable':'false','derived':'false','eContainingClass':'Graph','containment':'true','container':'false','resolveProxies':'false','eOpposite':'graph','eReferenceType':'Decorator'}}"" ""{'changeType': 'Preserve', 'type': 'object', 'className': 'EReference', 'attributes': {'id': '_XMyS1d60Eei97MD7GK1RmA', 'name':'graph','ordered':'true','unique':'true','lowerBound':'0','upperBound':'1','many':'false','required':'false','eType':'Graph','eGenericType':'org.eclipse.emf.ecore.impl.EGenericTypeImpl@26d6f4e0 (expression: Graph)','changeable':'true','volatile':'false','transient':'true','unsettable':'false','derived':'false','eContainingClass':'Decorator','containment':'false','container':'true','resolveProxies':'true','eOpposite':'decorators','eReferenceType':'Graph'}}""
# e 2 1 ""{'changeType': 'Remove', 'type': 'reference', 'referenceTypeName': 'eOpposite'}"" _ ""{'changeType': 'Preserve', 'type': 'object', 'className': 'EReference', 'attributes': {'id': '_XMyS1d60Eei97MD7GK1RmA', 'name':'graph','ordered':'true','unique':'true','lowerBound':'0','upperBound':'1','many':'false','required':'false','eType':'Graph','eGenericType':'org.eclipse.emf.ecore.impl.EGenericTypeImpl@35c48d76 (expression: Graph)','changeable':'true','volatile':'false','transient':'true','unsettable':'false','derived':'false','eContainingClass':'Decorator','containment':'false','container':'true','resolveProxies':'true','eOpposite':'decorators','eReferenceType':'Graph'}}""

# $$
# ---
# t # 195
# e 1 2 ""{'changeType': 'Change', 'type': 'attribute'}"" ""{'changeType': 'Preserve', 'type': 'object', 'className': 'EAttribute', 'attributes': {'id': '_lTgRnt6wEei97MD7GK1RmA', 'name':'parameterTypes','ordered':'true','unique':'true','lowerBound':'0','upperBound':'1','many':'false','required':'false','eType':'TypeArray','eGenericType':'org.eclipse.emf.ecore.impl.EGenericTypeImpl@219cff5d (expression: TypeArray)','changeable':'true','volatile':'false','transient':'false','unsettable':'false','derived':'false','eContainingClass':'BFunction','iD':'false','eAttributeType':'TypeArray'}}"" ""{'changeType': 'Change', 'type': 'attributeValue', 'className': 'EAttribute', 'attributeName': 'transient', 'valueBefore': 'false', 'valueAfter': 'true'}""
# e"," 1 0 ""{'changeType': 'Change', 'type': 'attribute'}"" _ ""{'changeType': 'Change', 'type': 'attributeValue', 'className': 'EAttribute', 'attributeName': 'derived', 'valueBefore': 'false', 'valueAfter': 'true'}""

# $$",3379,325,3704,7,1,1,['Change_attribute'],3351,63,"1 0 ""{'changeType': 'Change', 'type': 'attribute'}"" _ ""{'changeType': 'Change', 'type': 'attributeValue', 'className': 'EAttribute', 'attributeName': 'volatile', 'valueBefore': 'false', 'valueAfter': 'true'}""",True,False,True,True


# Given a csv file with the above format, this script will extract the completion and the completion_string from the csv
dataset = pd.read_csv('results_gpt_completion.csv')

completion_ground_truth = dataset['completion']
completion_generated = dataset['completion_string']

# next if available, we check how ofteh the completion is of 'changeType': 'Add'  and 'type': 'object' (i.e., ``adding a new class'').
is_add_object = completion_ground_truth.str.contains("'changeType': 'Add', 'type': 'object'").sum()
print(f'Relative amount of completions that are of type "Add object": {is_add_object / len(completion_ground_truth)}')
print(f'Absolute amount of completions that are of type "Add object": {is_add_object}')

Relative amount of completions that are of type "Add object": 0.23076923076923078
Absolute amount of completions that are of type "Add object": 51


# Evaluate "Predict Concept"

In [3]:
def get_name_or_key(full_string: str, identifier: str = ['name', 'key']):
    # regex for every identifier in the identifier list "'({identifier1}|{identifier2}|....})':'(\w+)'"
    regex = f"'({'|'.join(identifier)})':'(\w+)'"
    # map to regex
    matches = re.findall(regex, full_string)
    # get second group 
    assert len(matches) > 0, f'No matches found for {identifier} in {full_string}'
    assert len(matches) < 2, f'More than one match found for {identifier} in {full_string}'
    return matches[0][1]

def get_simple_name(class_name: str | float | None, name: float | str | None):
    if class_name is None or name is None or type(name) == float or type(class_name) == float:
        return None
    
    return class_name + '.' + name


In [4]:
# Slect only the add object case
is_add_object = completion_ground_truth.str.contains("'changeType': 'Add', 'type': 'object'")
# detect the not add object completions in generated
is_add_object_generated = completion_generated.str.contains("'changeType': 'Add', 'type': 'object'")
# replace the NaN values with False
is_add_object_generated = is_add_object_generated.fillna(False)
# print how many of the add object in ground truth are not add object in generated
is_add_object_gt_but_not_generated = is_add_object[is_add_object & ~is_add_object_generated].sum()
print(f'Relative amount of completions that are of type "Add object" in the ground truth completions, but not in the generated completions: {is_add_object_gt_but_not_generated / len(completion_ground_truth)}')
# select only the cases where the completion is of type 'Add object' in the ground truth completions and generated completions
is_add_object = is_add_object & is_add_object_generated


# map to boolean index, which we can use to select these cases
index = is_add_object[is_add_object].index


completion_ground_truth_filtered = completion_ground_truth[index]
completion_generated_filtered = completion_generated[index]


# check if the completion_generated is also of the same type
is_add_object_generated = completion_generated_filtered.str.contains("'changeType': 'Add', 'type': 'object'").sum()
print(f'Relative amount of completions that are of type "Add object" in the generated completions: {is_add_object_generated / len(completion_generated)}')

# Next we check if the generated completion className and name are the same as the ground truth completion className and name
classNames_ground_truth = completion_ground_truth_filtered.str.extract(r"'className': '(\w+)'")
classNames_generated = completion_generated_filtered.str.extract(r"'className': '(\w+)'")
names_ground_truth = completion_ground_truth_filtered.apply(lambda completion: get_name_or_key(completion))
names_generated = completion_generated_filtered.apply(lambda completion: get_name_or_key(completion))

# check when they are the same
same_className = classNames_ground_truth == classNames_generated
same_name = names_ground_truth == names_generated
print(f'Relative amount of completions where the className is the same: {same_className.sum() / len(same_className)}')
print(f'Absolute amount of completions where the className is the same: {same_className.sum()}')
print(f'Relative amount of completions where the name is the same: {same_name.sum() / len(same_name)}')
print(f'Absolute amount of completions where the name is the same: {same_name.sum()}')


# concate the same_name information to the original dataframe
dataset['same_name'] = same_name
dataset['same_class'] = same_className
dataset['completion_class_name_gt'] = classNames_ground_truth
dataset['completion_class_name_gen'] = classNames_generated
dataset['completion_name_gt'] = names_ground_truth
dataset['completion_name_gen'] = names_generated

# we compute a derived string in the form of 'className_name'
dataset['simplified_gt'] = dataset.apply(lambda row: get_simple_name(row['completion_class_name_gt'], row['completion_name_gt']), axis=1)
dataset['simplified_gen'] = dataset.apply(lambda row: get_simple_name(row['completion_class_name_gen'], row['completion_name_gen']), axis=1)

# for all names that are not the same, print (on by one) the ground truth and generated name, and the full completion strings
for i, row in dataset.iterrows():
    if row['same_name'] == False:
        print(f'Ground truth completion: {row["completion"]}')
        print(f'Generated completion: {row["completion_string"]}')
        print('')
        
# save the dataset with the additional information
dataset.to_csv('results_iso_check_with_info.csv')

Relative amount of completions that are of type "Add object" in the ground truth completions, but not in the generated completions: 0.004524886877828055
Relative amount of completions that are of type "Add object" in the generated completions: 0.22624434389140272
Relative amount of completions where the className is the same: 0    0.96
dtype: float64
Absolute amount of completions where the className is the same: 0    48
dtype: int64
Relative amount of completions where the name is the same: 0.98
Absolute amount of completions where the name is the same: 49
Ground truth completion:  1 2 "{'changeType': 'Add', 'type': 'reference', 'referenceTypeName': 'eOperations'}" _ "{'changeType': 'Add', 'type': 'object', 'className': 'EOperation', 'attributes': {'id': '_lU7gFt6tEei97MD7GK1RmA', 'name':'getStereotypeEnd','ordered':'false','unique':'true','lowerBound':'0','upperBound':'1','many':'false','required':'false','eType':'Property','eGenericType':'org.eclipse.emf.ecore.impl.EGenericTypeImpl@

# Evaluate 'Predict Associations'
TO evaluate predict associations, we first check if they are between the right nodes (corresponding to classes) and if they are of correct type.
TODO actually here we could check for more than only the "Add_node" type completions. 

In [13]:
def get_reference_identifier(full_string: str):
    """Extract an identifier source_id.target_id.type.referenceTypeName from a string

    Args:
        full_string (str): The edge completion
    """
    regex = r".*(\d+) (\d+) \"{'changeType': '(\w+)', 'type': '(\w+)', 'referenceTypeName': '(\w+)'}\""
    matches = re.findall(regex, full_string)
    assert len(matches) > 0, f'No matches found for {full_string}'
    assert len(matches) < 2, f'More than one match found for {full_string}'
    return f"{matches[0][0]}_{matches[0][1]}_{matches[0][3]}_{matches[0][4]}"
    

In [14]:
edges_ground_truth = completion_ground_truth_filtered.apply(lambda completion: get_reference_identifier(completion))
edges_generated = completion_generated_filtered.apply(lambda completion: get_reference_identifier(completion))

# check when they are the same
same_edges = edges_ground_truth == edges_generated
# add all to the dataframe
dataset['same_edges'] = same_edges
dataset['edges_gt'] = edges_ground_truth
dataset['edges_gen'] = edges_generated

# print the cases when the edges are not the same
for i, row in dataset.iterrows():
    if row['same_edges'] == False:
        print(f'Ground truth completion: {row["completion"]}')
        print(f'Generated completion: {row["completion_string"]}')
        print('')
        
# save the dataset with the additional information
dataset.to_csv('results_iso_check_with_info.csv')

Ground truth completion:  0 1 "{'changeType': 'Add', 'type': 'reference', 'referenceTypeName': 'eParameters'}" _ "{'changeType': 'Add', 'type': 'object', 'className': 'EParameter', 'attributes': {'id': '_ewiMaN6zEei97MD7GK1RmA', 'name':'unit','ordered':'true','unique':'true','lowerBound':'0','upperBound':'1','many':'false','required':'false','eType':'BuildUnit','eGenericType':'org.eclipse.emf.ecore.impl.EGenericTypeImpl@20c26698 (expression: BuildUnit)','eOperation':'getEffectiveBuildUnit'}}"

$$
Generated completion: 2 1 "{'changeType': 'Add', 'type': 'reference', 'referenceTypeName': 'eParameters'}" _ "{'changeType': 'Add', 'type': 'object', 'className': 'EParameter', 'attributes': {'id': '_ewiMZd6zEei97MD7GK1RmA', 'name':'unit','ordered':'true','unique':'true','lowerBound':'0','upperBound':'1','many':'false','required':'false','eType':'BuildUnit','eGenericType':'org.eclipse.emf.ecore.impl.EGenericTypeImpl@11a034f8 (expression: BuildUnit)','eOperation':'getEffectiveBuildUnit'}}"

Gro